In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
from stratosphere import Stratosphere
from stratosphere.utils.inspect_flows import InspectFlows
from stratosphere import options
from stratosphere.services.extractor import Extractor
from stratosphere.storage.models import Flow
from stratosphere.services.extractors.dummy import dummy_uuid
from stratosphere.services.extractors.dummy import add_dummy

In [3]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100

[[Home](/)]
[[Webapps](/jupyter/voila/tree/webapps)]

# Entity overview



In [4]:
add_dummy()
kb = Stratosphere(options.get("db.url_kb"))

In [15]:
import os
from urllib.parse import parse_qs

params = parse_qs(os.environ.get("QUERY_STRING"), keep_blank_values=True)

if 'entity_id' not in params:
    params['entity_id'] =[dummy_uuid]


In [17]:
entity_id = params['entity_id'][0]

In [22]:
def get_entity(entity_id):
    
    q = f"""
        SELECT
            *
        FROM entities
        WHERE entity_id = "{entity_id}"
    """
    df = kb.db.pandas(q)
    
    
    return df.fillna("")

#get_entity(entity_id)

,entity_id,group,type,data,blob,timestamp
0,275876e34cf609db118f3d84b799a790,dummy,dummy,{},,2023-04-12 12:11:21.360329


## Properties

In [26]:
import json
from IPython.display import HTML
from markdown import markdown

def render_entity(entity_id):
    
    mks= []

    for key, value in get_entity(entity_id).iloc[0].to_dict().items():    
        if key == "blob":
            continue
        if key == 'data':
            mks.append("* **data**")
            data = json.loads(value)
            for data_key, data_value in data.items():
                mks.append(f'    * **{data_key}**: {data_value}')                    
        else:
            mks.append(f'* **{key}**: {value}')

    html = markdown('\n'.join(mks))
    display(HTML(html))

render_entity(entity_id)

## Relationships

In [30]:
def render_href(href_col):    
    if not href_col[1]:
        return ""
    return f'<a target="_blank" href={href_col[1]}>{href_col[0]}</a>'


def render_df(df):
    if len(df) == 0:
        print('No matches.')
        return
    
    df = df.copy()

    df['src'] =  df.apply(lambda row: [row.src, f'/jupyter/voila/render/webapps/entity-overview.ipynb?entity_id={row.src}'], axis=1)
    df['dst'] =  df.apply(lambda row: [row.dst, f'/jupyter/voila/render/webapps/entity-overview.ipynb?entity_id={row.dst}'], axis=1)
    
    df = df.fillna("")
    
    display(HTML(df.to_html(escape=False, formatters=dict(src=render_href, dst=render_href)))) # escape=False ,formatters=dict(url_picture=render_img_url))
    

In [31]:
def get_relationships(entity_id):
    
    q = f"""
        SELECT
            *
        FROM relationships
        WHERE dst = "{entity_id}" or src = "{entity_id}"
    """
    df = kb.db.pandas(q)
    
    return df

df = get_relationships(entity_id)
df.fillna("")

render_df(df)

,relationship_id,group,type,src,dst,data,blob,timestamp
0,275876e34cf609db118f3d84b799a790,dummy,dummy,src,src,{},,2023-04-12 12:15:48.573458
